<a href="https://colab.research.google.com/github/lextale/IonianDevsBitch/blob/master/PC_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DEBUG_LOG = ''

In [ ]:
products = {'RAM': {'Id': 'a07Qy00000BarGcIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/38503320/G-Skill-Trident-Z5-Neo-RGB-32GB-DDR5-RAM-me-2-Modules-2x16GB-kai-Tachytita-6000-gia-Desktop-F5-6000J3038F16GX2-TZ5NR.html'},
              'PSU': {'Id': 'a07Qy00000BarGdIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/28883751/Corsair-RMx-Series-RM750x-2021-750W-Mayro-Trofodotiko-Ypologisti-Full-Modular-80-Plus-Gold.html'},
              '1TB': {'Id': 'a07Qy00000BarGeIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/26796338/Samsung-870-Evo-SSD-1TB-2-5-SATA-III-MZ-77E1T0B-EU.html'},
              'CPU FAN': {'Id': 'a07Qy00000BarGfIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/14956693/Be-Quiet-Dark-Rock-4-PSyktra-Epexergasti-Diplou-Anemistira-gia-Socket-AM4-AM5-115x.html'},
              'MOTHERBOARD': {'Id': 'a07Qy00000BarGgIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/39030342/Asus-ROG-Strix-B650E-F-Gaming-Wi-Fi-Motherboard-ATX-me-AMD-AM5-Socket.html'},
              'GPU': {'Id': 'a07Qy00000BarGhIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/29549346/Gigabyte-GeForce-RTX-3060-12GB-GDDR6-Gaming-OC-rev-2-0-Karta-Grafikon-GV-N3060GAMING-OC-12GD-2-0.html'},
              'CPU': {'Id': 'a07Qy00000BarGiIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/37822611/AMD-Ryzen-7-7700X-4-5GHz-Epexergastis-8-Pyrinon-gia-Socket-AM5-se-Kouti.html'},
              '500GB': {'Id': 'a07Qy00000BarGjIAJ', 'SkroutzProductURL__c': 'https://www.skroutz.gr/s/25294605/Samsung-980-Pro-SSD-500GB-M-2-NVMe-PCI-Express-4-0-MZ-V8P500BW.html'}
              }



In [ ]:
def getPriceGraphLink(productLink):
  if (productLink.endswith('.html')):
    return productLink[0:-5]+'/price_graph.json'
  return productLink+'/price_graph.json'

In [ ]:
from json.decoder import JSONDecodeError
import json

def getPriceGraphJSON(priceGraphLink):
  global DEBUG_LOG
  # Injecting the Python variable into the shell command using {}
  rawJSON = !curl --compressed {priceGraphLink} \
    -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" \
    -H "Accept-Language: el,en;q=0.9,en-GB;q=0.8,en-US;q=0.7" \
    -H "Cache-Control: max-age=0" \
    -H "Cookie: __Secure-ASPSESSION=evtqsnumrxf2mht34ky4bg3p; cookiesession1=678B287EKLMNOPQRSTUV12345678848E; agreeToCookies=0" \
    -H "Sec-Fetch-Dest: document" \
    -H "Sec-Fetch-Mode: navigate" \
    -H "Sec-Fetch-Site: none" \
    -H "Sec-Fetch-User: ?1" \
    -H "Upgrade-Insecure-Requests: 1" \
    -A "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"

  # Join lines of output into a single string
  rawJSON = "\n".join(rawJSON)

  # Attempt to parse JSON if possible
  try:
    json_data = json.loads(rawJSON)
    return json_data
  except json.JSONDecodeError:
    DEBUG_LOG += "-----------------------------------------\n"
    DEBUG_LOG += "Error: Unable to parse JSON\n"
    DEBUG_LOG += "priceGraphLink: ", priceGraphLink, "\n"
    DEBUG_LOG += "Output: ", rawJSON, "\n"
    DEBUG_LOG += "-----------------------------------------\n"
    return json.loads({})
  except AttributeError as e:
    DEBUG_LOG += "-----------------------------------------\n"
    DEBUG_LOG += f"Error: {e}\n"
    DEBUG_LOG += "priceGraphLink: ", str(priceGraphLink), "\n"
    DEBUG_LOG += "Output: ", rawJSON, "\n"
    DEBUG_LOG += "-----------------------------------------\n"
    return json.loads({})
  except TypeError as e:
    DEBUG_LOG += "-----------------------------------------\n"
    DEBUG_LOG += f"Error: {e}\n"
    DEBUG_LOG += "priceGraphLink: ", str(priceGraphLink), "\n"
    DEBUG_LOG += "Output: ", rawJSON, "\n"
    DEBUG_LOG += "-----------------------------------------\n"
    return json.loads({})
  except:
    DEBUG_LOG += "-----------------------------------------\n"
    DEBUG_LOG += "Error\n"
    DEBUG_LOG += "priceGraphLink: ", str(priceGraphLink), "\n"
    DEBUG_LOG += "Output: ", rawJSON, "\n"
    DEBUG_LOG += "-----------------------------------------\n"
    return json.loads({})

In [ ]:
from datetime import datetime

# Merging dictionaries based on timestamp
def mergeByTimestamp(productMinPrice, productPopularity):
    # Create a new list to store the merged result
    merged_values = []

    # Convert productPopularity to a dictionary indexed by timestamp for quick lookup
    popularity_dict = {item['timestamp']: item['value'] for item in productPopularity['values']}

    # Loop through productMinPrice and merge with productPopularity if the timestamp matches
    for price_item in productMinPrice['values']:
        timestamp = price_item['timestamp']
        popularity_value = popularity_dict.get(timestamp, None)  # Get popularity value if it exists

        # Add the popularity value to the price_item
        merged_item = {
            'Name': timestamp,
            #'date': datetime.fromtimestamp(timestamp).strftime('%d-%m-%Y %H:%M:%S'),
            'SkroutzProductPrice__c': price_item['value'],
            'SkroutzProductPopularity__c': popularity_value  # None if no match found
        }
        merged_values.append(merged_item)

    return merged_values

In [ ]:
for pcpart, partinfo in products.items():
  productlink = partinfo['SkroutzProductURL__c']
  graphJSON = getPriceGraphJSON(getPriceGraphLink(productlink))
  productMinPrice = graphJSON['min_price']['graphData']['all']
  productPopularity = graphJSON['popularity']['graphData']['all']
  timestampsList = mergeByTimestamp(productMinPrice, productPopularity)
  products[pcpart]['SkroutzPriceGraph__c'] = graphJSON
  products[pcpart]['timestampsList'] = timestampsList
  products[pcpart]['SkroutzMinPrice__c'] = graphJSON['min_price']['min']
  products[pcpart]['SkroutzMaxPrice__c'] = graphJSON['min_price']['max']
  products[pcpart]['SkroutzMinPopularity__c'] = graphJSON['popularity']['min']
  products[pcpart]['SkroutzMaxPopularity__c'] = graphJSON['popularity']['max']

In [ ]:
def getCommonTimestamps(products):
  return [timestamp['timestamp'] for sub_dict in products.values() for timestamp in sub_dict['graphJSON']['stats'] if timestamp['timestamp'] not in b_values]

In [ ]:
productsToUpsert = []
for pcpart in products.keys():
  productsToUpsert.append({
      'Id': products[pcpart]['Id'],
      'SkroutzProductURL__c': products[pcpart]['SkroutzProductURL__c'],
      'SkroutzPriceGraph__c': str(products[pcpart]['SkroutzPriceGraph__c']),
      'SkroutzMinPrice__c': products[pcpart]['SkroutzMinPrice__c'],
      'SkroutzMaxPrice__c': products[pcpart]['SkroutzMaxPrice__c'],
      'SkroutzMinPopularity__c': products[pcpart]['SkroutzMinPopularity__c'],
      'SkroutzMaxPopularity__c': products[pcpart]['SkroutzMaxPopularity__c']
  })

In [ ]:
timestampToUpsert = []
for pcpart in list(products.keys()):
  id = products[pcpart]['Id']
  for timestamp in products[pcpart]['timestampsList']:
    tmsName = timestamp['Name']
    price = timestamp['SkroutzProductPrice__c']
    popularity = timestamp['SkroutzProductPopularity__c']
    timestampToUpsert.append({'ProductId__c': id,
                            'Name': tmsName,
                            'SkroutzProductPopularity__c': popularity,
                            'SkroutzProductPrice__c': price
                            })

In [ ]:
for i, ts in enumerate(timestampToUpsert):
  if (ts['SkroutzProductPrice__c'] == 0):
    timestampToUpsert.pop(i)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(productsToUpsert)
df.to_excel('products.xlsx', index=False)
#df.to_csv('products.csv', index=False)
df.drop('Id', axis=1).drop('SkroutzPriceGraph__c', axis=1).to_csv('products.csv', index=False)

In [ ]:
df = pd.DataFrame(timestampToUpsert)
df.to_excel('timestamps.xlsx', index=False)
df.to_csv('timestamps.csv', index=False)

In [ ]:
df = pd.DataFrame(productsToUpsert)
df.to_excel('products.xlsx', index=False)
df.drop('SkroutzProductURL__c', axis=1).drop('SkroutzPriceGraph__c', axis=1).to_excel('products.xlsx', index=False)

In [ ]:
!cat /content/products.csv

Id,SkroutzMinPrice__c,SkroutzMaxPrice__c,SkroutzMinPopularity__c,SkroutzMaxPopularity__c
a07Qy00000BarGcIAJ,"119,15","515,48",174,2866
a07Qy00000BarGdIAJ,"106,33","152,27",533,2268
a07Qy00000BarGeIAJ,"53,52","101,0",1035,4805
a07Qy00000BarGfIAJ,"57,97","72,99",159,521
a07Qy00000BarGgIAJ,"264,91","381,9",7,726
a07Qy00000BarGhIAJ,"9,35","443,0",1747,8098
a07Qy00000BarGiIAJ,"272,73","519,9",765,2181
a07Qy00000BarGjIAJ,"50,0","88,49",398,3299


In [ ]:
!cat /content/timestamps.csv

ProductId__c,Name,SkroutzProductPopularity__c,SkroutzProductPrice__c
a07Qy00000BarGcIAJ,1663880400,0,0.0
a07Qy00000BarGcIAJ,1663966800,0,0.0
a07Qy00000BarGcIAJ,1664053200,0,0.0
a07Qy00000BarGcIAJ,1664139600,0,0.0
a07Qy00000BarGcIAJ,1664226000,0,0.0
a07Qy00000BarGcIAJ,1664312400,0,0.0
a07Qy00000BarGcIAJ,1664398800,0,0.0
a07Qy00000BarGcIAJ,1664485200,0,0.0
a07Qy00000BarGcIAJ,1664571600,0,0.0
a07Qy00000BarGcIAJ,1664658000,0,0.0
a07Qy00000BarGcIAJ,1664744400,0,0.0
a07Qy00000BarGcIAJ,1664830800,0,379.65
a07Qy00000BarGcIAJ,1664917200,0,0.0
a07Qy00000BarGcIAJ,1665003600,0,0.0
a07Qy00000BarGcIAJ,1665090000,0,0.0
a07Qy00000BarGcIAJ,1665176400,0,0.0
a07Qy00000BarGcIAJ,1665262800,0,0.0
a07Qy00000BarGcIAJ,1665349200,2866,381.15
a07Qy00000BarGcIAJ,1665435600,2866,0.0
a07Qy00000BarGcIAJ,1665522000,2866,372.49
a07Qy00000BarGcIAJ,1665608400,2866,359.3
a07Qy00000BarGcIAJ,1665694800,2866,368.52
a07Qy00000BarGcIAJ,1665781200,2866,368.52
a07Qy00000BarGcIAJ,1665867600,2866,368.52
a07Qy00000BarGcIAJ,1665954